In [1]:
import numpy as np
import tensorflow as tf

from datetime import datetime

In [2]:
mnist = input_data.read_data_sets("/tmp/data")

_, n = mnist.train.images.shape

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
rootLogDir = "tfLogs"
logDir = "{0}/run-{1}/".format(rootLogDir, now)

fileWriter = tf.summary.FileWriter( logDir, tf.get_default_graph() )

In [4]:
def mnistClassifier( X, y, nOut, nl = 1, nh = 100, alpha = 0.01, momentum = 0.9 ):

    if ( nl < 1 ):
        print( "You need at least one hidden layer." )
        return

    if ( nh < 1 ):
        print( "you need at least one neuron." )
        return

    with tf.name_scope( "dnn" ):
        layers = [ tf.layers.dense( X, nh, name = "hidden1", activation = tf.nn.relu ) ]

        for i in range(2, nl + 1):
            layers.append( tf.layers.dense( layers[-1], nh, name = "hidden" + str(i), activation = tf.nn.relu ) )

        logits = tf.layers.dense( layers[-1], nOut, name = "output" )

    with tf.name_scope("loss"):
        crossEnt = tf.nn.sparse_softmax_cross_entropy_with_logits( labels = y, logits = logits)
        loss = tf.reduce_mean( crossEnt, name = "loss" )
        
    with tf.name_scope("eval"):
        correct = tf.nn.in_top_k(logits, y, 1)
        accuracy = tf.reduce_mean( tf.cast(correct, tf.float32) )
        
    with tf.name_scope("train"):
        opt = tf.train.MomentumOptimizer( learning_rate = alpha, momentum = momentum)
        training = opt.minimize( loss )
        lossSummary = tf.summary.scalar("crossEntropy", loss)
        
    with tf.name_scope("utility"):
        init = tf.global_variables_initializer()
        saver = tf.train.Saver()
        
    return loss, training, accuracy, lossSummary, init, saver


In [5]:
X = tf.placeholder(tf.float32, shape = (None, n), name = "X")
y = tf.placeholder(tf.int32, shape = (None), name = "y")

loss, training, accuracy, lossSummary, init, saver = mnistClassifier( X, y, 10,
                                                                      nl = 4,
                                                                      nh = 200,
                                                                      alpha = 0.01,
                                                                      momentum = 0.9 )

In [6]:
nEpochs = 1000
batchSize = 64 #2048

hiVal = 0
patience = 0

with tf.Session() as sess:

    init.run()
    
    for epoch in range(nEpochs):
        for i in range( mnist.train.num_examples // batchSize ):
            
            batchX ,batchY = mnist.train.next_batch( batchSize )
            sess.run( training, feed_dict = { X : batchX, y : batchY } )
            
        trainAcc = accuracy.eval( feed_dict = { X : batchX, y : batchY } )
        valAcc   = accuracy.eval( feed_dict = { X : mnist.validation.images,
                                                y : mnist.validation.labels } )

        print( epoch, "Training:", trainAcc, "Validation:", valAcc )

        if ( valAcc > hiVal ):
            hiVal = valAcc
            patience = 0

        else:
            patience += 1

        if ( patience >= 10):
            print("No imporvement on validation set after {0} epochs. Training competed".format(patience))
            break
            
    print("saving model.")
    saver.save(sess, "./model.ckpt")


0 Training: 0.96875 Validation: 0.957
1 Training: 1.0 Validation: 0.9632
2 Training: 0.984375 Validation: 0.9734
3 Training: 1.0 Validation: 0.976
4 Training: 1.0 Validation: 0.9784
5 Training: 0.984375 Validation: 0.9784
6 Training: 1.0 Validation: 0.9804
7 Training: 0.96875 Validation: 0.9806
8 Training: 1.0 Validation: 0.9826
9 Training: 1.0 Validation: 0.9796
10 Training: 0.984375 Validation: 0.9774
11 Training: 0.984375 Validation: 0.9836
12 Training: 1.0 Validation: 0.979
13 Training: 1.0 Validation: 0.9786
14 Training: 1.0 Validation: 0.9812
15 Training: 0.984375 Validation: 0.9778
16 Training: 1.0 Validation: 0.981
17 Training: 1.0 Validation: 0.9828
18 Training: 1.0 Validation: 0.9848
19 Training: 1.0 Validation: 0.9842
20 Training: 1.0 Validation: 0.985
21 Training: 1.0 Validation: 0.9846
22 Training: 1.0 Validation: 0.9848
23 Training: 1.0 Validation: 0.9848
24 Training: 1.0 Validation: 0.9846
25 Training: 1.0 Validation: 0.985
26 Training: 1.0 Validation: 0.985
27 Training:

In [7]:
tf.reset_default_graph()
#sess = tf.Session()

X = tf.placeholder(tf.float32, shape = (None, n), name = "X")
y = tf.placeholder(tf.int32, shape = (None), name = "y")

loss, training, accuracy, lossSummary, init, saver = mnistClassifier( X, y, 10,
                                                                      nl = 4,
                                                                      nh = 200,
                                                                      alpha = 0.01,
                                                                      momentum = 0.9 )

with tf.Session() as sess:

    saver.restore( sess, "./model.ckpt" )
    testAcc = accuracy.eval( feed_dict = { X : mnist.test.images, y : mnist.test.labels })

    print( "Accuracy on test set:", testAcc )

INFO:tensorflow:Restoring parameters from ./model.ckpt
Accuracy on test set: 0.9823
